# JV Analysis tool

## 1 Select Upload
Select one or multiple uploads

In [4]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import os
import sys
from main import load_files, find_unique_values
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd
import ipywidgets as widgets
import numpy as np

sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_sample_description, get_all_JV, get_all_measurements_except_JV

url_base ="https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

batch_ids_list_tmp = list(get_batch_ids(url, token))
batch_ids_list = []
for b in batch_ids_list_tmp:
    if "_".join(b.split("_")[:-1]) in batch_ids_list_tmp:
        continue
    batch_ids_list.append(b)

batch_ids = widgets.SelectMultiple(options=batch_ids_list,description='Batches',  layout=widgets.Layout(width='800px', height='80px'))

# Global widgets and variables
button = widgets.Button(description="Load Data",layout=widgets.Layout(min_width='100px'))
warning_sign = "\u26A0"


search_field = widgets.Text(description="Filter")
search = widgets.Button(description="Search",layout=widgets.Layout(min_width='100px'))
folders = os.listdir("..")

path = widgets.SelectMultiple(
    options=folders,
    description='Method',
    disabled=False
)
out = widgets.Output()
out2 = widgets.Output()
read = widgets.Output()
dynamic_content = widgets.Output()  # For dynamically updated content
results_content = widgets.Output(layout={
    # 'border': '1px solid black',  # Optional: adds a border to the widget
    'width': '400px',  # Set the width
    'height': '300px',  # Set the height
    'overflow': 'scroll',  # Adds a scrollbar if content overflows
    })

default_variables = widgets.Dropdown(
    options=['all', 'Batch name', 'Variation'],
    value='all',
    description='Defaults:',
    disabled=False,
)

data = {}
is_conditions = False
unique_vals = []

# Function to create widgets based on elements list
def create_widgets_table(elements_list):
    rows = []
    text_widgets = {}  # Dictionary to store text strings
    for item in elements_list:
        item_split = item.split("&")
        batch, variable = "", item
        if len(item_split) >=2:
            batch, variable = item_split[0], "&".join(item_split[1:])
        default_value = ""
        if default_variables.value == "Batch name":
            default_value = batch if batch else "_".join(item.split("_")[:-1])
        if default_variables.value == "Variation":
            default_value = variable
        label = widgets.Label(value=variable)
        text_input = widgets.Text(value=default_value, placeholder='Variable e.g. 1000 rpm')
        row = widgets.HBox([label, text_input])
        rows.append(row)
        text_widgets[item] = text_input  
    return widgets.VBox(rows), text_widgets



def get_jv_data_for_analysis(sample_ids):
    columns_jvc = ['Voc(V)','Jsc(mA/cm2)','FF(%)','PCE(%)','V_mpp(V)','J_mpp(mA/cm2)',
                   'P_mpp(mW/cm2)','R_series(Ohmcm2)','R_shunt(Ohmcm2)','sample','batch',
                   'condition','cell','direction','ilum']
    columns_cur = ['index','sample','batch','condition','variable','cell','direction','ilum']
    rows_jvc = []
    rows_cur = []
    all_jvs = get_all_JV(url, token,sample_ids)
    for sid in sample_ids:
        jv_res = all_jvs.get(sid,[])
        with out:
            out.clear_output()
            print("Processing: ", sid)
        for jv_data, jv_md in jv_res:
            for c in jv_data["jv_curve"]:
                file_name = os.path.join("..",jv_md["upload_id"],jv_data.get("data_file"))
                illum = "Dark" if "dark" in c["cell_name"].lower() else "Light"
                cell = c["cell_name"][0]
                direction = "Forward" if "for" in c["cell_name"].lower() else "Reverse"
                row = [c["open_circuit_voltage"],-c["short_circuit_current_density"],100*c["fill_factor"],c["efficiency"],c["potential_at_maximum_power_point"],-c["current_density_at_maximun_power_point"],
                       -c["potential_at_maximum_power_point"]*c["current_density_at_maximun_power_point"],c["series_resistance"],c["shunt_resistance"],file_name,file_name.split("/")[1],"w",
                        cell,direction, illum]
                rows_jvc.append(row)
                row_v = ["_".join(["Voltage (V)",cell,direction,illum]),file_name,file_name.split("/")[1],"w","Voltage (V)", 
                         cell,direction, illum]
                row_v.extend(c["voltage"])
                row_j = ["_".join(["Current Density(mA/cm2)",cell,direction,illum]),file_name,file_name.split("/")[1],"w","Current Density(mA/cm2)", 
                         cell,direction, illum]
                row_j.extend(c["current_density"])
                for i in range(len(row_v[8:])):
                    if i not in columns_cur:
                        columns_cur.append(i)

                rows_cur.append(row_v)
                rows_cur.append(row_j)
    df_jvc = pd.DataFrame(rows_jvc,columns=columns_jvc)
    df_cur = pd.DataFrame(rows_cur,columns=columns_cur)
    return df_jvc, df_cur

def on_button_clicked(b):
    global data
    data = {}
    dynamic_content.clear_output()
    with out:
        out.clear_output()
        print("Loading Data")

        sample_ids = get_ids_in_batch(url, token, batch_ids.value)
        identifiers = get_sample_description(url, token, sample_ids)              
        df_jvc, df_cur = get_jv_data_for_analysis(sample_ids)
        data["jvc"] = pd.concat([data.get("jvc", pd.DataFrame()), df_jvc], ignore_index=True)
        data["curves"] = pd.concat([data.get("curves", pd.DataFrame()), df_cur], ignore_index=True)
        out.clear_output()
        print("Data Loaded")
        data["jvc"]["subbatch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-2])
        data["jvc"]["batch"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-3])
        data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: '_'.join(x.split('/')[-1].split('_')[3:-1]))
        data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split(".")[0])
        if identifiers:
            data["jvc"]["identifier"] = data["jvc"]["identifier"].apply(lambda x: f'{"_".join(x.split("_")[:-1])}&{identifiers.get(x,"No variation specified")}')
        else:
            data["jvc"]["identifier"] = data["jvc"]["sample"].apply(lambda x: "_".join(x.split('/')[-1].split(".")[0].split("_")[:-1]))
            
                
            
        data["jvc"]["sample"] = data["jvc"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
        #data["curves"]["subbatch"] = data["curves"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-2])
        #data["curves"]["batch"] = data["curves"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[-3])
        #data["curves"]["identifier"] = data["curves"]["sample"].apply(lambda x: '_'.join(x.split('/')[-1].split('_')[3:-1]))
        data["curves"]["sample"] = data["curves"]["sample"].apply(lambda x: x.split('/')[-1].split('.')[0].split('_',4)[-1])
    df_jvc.to_csv("export_jvc.csv")
    df_cur.to_csv("export_cur.csv")
    
    #print(data["jvc"]["sample"])
    global unique_vals
    unique_vals = find_unique_values(data["jvc"])
    make_variables_menu(unique_vals)
    
def brief_data_summary(df):
    # Perform the calculations
    global_mean_PCE = df['PCE(%)'].mean()
    global_std_PCE = df['PCE(%)'].std()
    max_PCE_row = df.loc[df['PCE(%)'].idxmax()]
    mean_std_PCE_per_sample = df.groupby(['batch', 'sample'])['PCE(%)'].agg(['mean', 'std'])
    highest_mean_PCE_sample = mean_std_PCE_per_sample.idxmax()['mean']
    lowest_mean_PCE_sample = mean_std_PCE_per_sample.idxmin()['mean']
    highest_PCE_per_sample = df.loc[df.groupby(['sample'])['PCE(%)'].idxmax(), ['sample', 'cell', 'PCE(%)']]
    
    # Formatting the output as Markdown
    markdown_output = f"""
### Summary Statistics

**Global mean PCE(%)**: {global_mean_PCE:.2f} ± {global_std_PCE:.2f}%


|   | Sample | Mean PCE(%) | Std PCE(%) |
|---|--------|-------------|------------|
| Best sample | {highest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[highest_mean_PCE_sample, 'std']:.2f}% |
| Worst sample | {lowest_mean_PCE_sample[1]} | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'mean']:.2f}% | {mean_std_PCE_per_sample.loc[lowest_mean_PCE_sample, 'std']:.2f}% |

#### Highest PCE(%) per Sample

| | Sample | Cell | PCE(%) |
|-|--------|------|--------|
| Best Overall | {max_PCE_row['sample']} | {max_PCE_row['cell']} | {max_PCE_row['PCE(%)']:.2f}% |
"""
    for sample, row in highest_PCE_per_sample.set_index('sample').iterrows():
        markdown_output += f"| | {sample} | {row['cell']} | {row['PCE(%)']:.2f}% |\n"
        
    return markdown_output

def on_retrieve_clicked(text_widgets_dict):
    global is_conditions
    is_conditions = True
    conditions_dict = {}
    read.clear_output()
    for item, text_widget in text_widgets_dict.items():
        # print(item, text_widget.value)
        conditions_dict[item] = text_widget.value
    data['jvc']['condition'] = data['jvc']['identifier'].map(conditions_dict)
    with read:
        print("Variables loaded")

def make_variables_menu(unique_vals):
    variables_markdown = f"""
# 1a Add variable names
There are {len(unique_vals)} samples found.
If you tested specific variables or conditions for each sample, please write them down below.
""" 
    results_markdown = brief_data_summary(data['jvc'])
    with dynamic_content:
        display(Markdown(variables_markdown))
        display(default_variables)
        widgets_table, text_widgets_dict = create_widgets_table(unique_vals)
        retrieve_button = widgets.Button(description="Confirm variables",layout=widgets.Layout(min_width='150px'))
        retrieve_button.on_click(lambda b: on_retrieve_clicked(text_widgets_dict))
        information_group = widgets.HBox([widgets_table, results_content])
        display(information_group)
        button_group = widgets.HBox([retrieve_button, read])
        display(button_group)
    
    results_html = widgets.HTML(value=f"<div>{results_markdown}</div>")
    with results_content:
        results_content.clear_output() 
        display(Markdown(results_markdown))
    with read:
        read.clear_output()
        print(f"{warning_sign} Variables not loaded")
        
def on_change_default_variables(b):
    dynamic_content.clear_output()
    global unique_vals
    make_variables_menu(unique_vals)

default_variables.observe(on_change_default_variables,names=['value'])
       

# Bind the 'Load Data' button click event
button.on_click(on_button_clicked)

# Function to handle search button click, filters the options based on search_field value
def on_search_clicked(b):  
    if not search_field.value:  
        batch_ids.options = batch_ids_list  
        return  

    search_term = search_field.value.lower()  # Convert search term to lowercase  
    batch_ids.options = [d for d in batch_ids_list if search_term in d.lower()]  #

# Bind the 'Search' button click event
search.on_click(on_search_clicked)

# Display the initial UI components
load_group = widgets.HBox([button, out])
# display(widgets.VBox([search_field, search, path, load_group]))
display(widgets.VBox([search_field, search, batch_ids, load_group]))
display(dynamic_content)  # This will be updated dynamically with the variables menu


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Output()

# Quick download
Click here if you want to download the data as  csv tables. The `jvc.csv` table contains the derived quantities and the `curves.csv` tables contains the JV curves.


In [2]:
from IPython.display import HTML, clear_output
from base64 import b64encode
import io

quick_download_content = widgets.Output()
display(quick_download_content)
def trigger_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with quick_download_content:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = widgets.Button(description='Download JV',layout=widgets.Layout(min_width='150px'))
def download_shit(e=None):
    jvc = io.StringIO()
    curves = io.StringIO()
    data["jvc"].to_csv(jvc)
    data["curves"].to_csv(curves)
    trigger_download( jvc.getvalue(), 'jvc.csv', kind='text/plain')
    trigger_download( curves.getvalue(), 'curves.csv', kind='text/plain')
btn.on_click(download_shit)
display(btn)

Output()

Button(description='Download JV', layout=Layout(min_width='150px'), style=ButtonStyle())

# Overview other measurements

In [3]:
show_other_measurements = widgets.Output()
display(show_other_measurements)

btn = widgets.Button(description='Show Measurements',layout=widgets.Layout(min_width='150px'))
def show_table(e=None):
    sample_ids = get_ids_in_batch(url, token, batch_ids.value)
    data = get_all_measurements_except_JV(url, token, sample_ids)

    df = pd.DataFrame()
    def make_clickable(r):
        if "SEM" in r[1]["entry_type"]:
            return '<a href="{}" rel="noopener noreferrer" target="_blank" >{}</a>' \
                    .format(f'{url_base}/nomad-oasis/gui/entry/id/{r[1]["entry_id"]}/data/data/images:0/image_preview/preview',r[1]["entry_type"].split("_")[-1])
        return '<a href="{}" rel="noopener noreferrer" target="_blank" >{}</a>'.format(f'{url_base}/nomad-oasis/gui/entry/id/{r[1]["entry_id"]}/data/data',r[1]["entry_type"].split("_")[-1])

    for key, value in data.items():
        df[key] = pd.Series([make_clickable(r) for r in value])
    with show_other_measurements:
        clear_output()
        display(HTML(df.to_html(escape=False)))

btn.on_click(show_table)
display(btn)

Output()

Button(description='Show Measurements', layout=Layout(min_width='150px'), style=ButtonStyle())

In [4]:
from main import data_filter_setup
#import ipywidgets as widgets
#from IPython.display import display, Markdown

# Define warning sign variable
warning_sign = "⚠️"

# Placeholder for presets data structure
filter_presets = {
    "Default": [("PCE(%)", "<", "40"), ("FF(%)", "<", "89"), ("FF(%)", ">", "24"), ("Voc(V)", "<", "2"), ("Jsc(mA/cm2)", ">", "-30")],
    "Preset 2": [("FF(%)", "<", "15"), ("PCE(%)", ">=", "10")]
}

# Initialize all widgets first before any callbacks
dropdown_width = '80px'
text_input_width = '100px'
button_width = '100px'

# Initialize main widgets
main_output_content = widgets.Output(layout={
    'width': '400px',
    'height': '250px',
    'overflow': 'scroll',
})

# Initialize preset dropdown first
preset_dropdown_filter = widgets.Dropdown(
    options=list(filter_presets.items()),
    description='Filters',
    layout=widgets.Layout(width='200px')
)

# Initialize buttons
add_button_filter = widgets.Button(description="Add Filter")
remove_button_filter = widgets.Button(description="Remove Filter")
#save_button_filter = widgets.Button(description="Save Filter", layout=widgets.Layout(width=button_width))
#save_name_filter = widgets.Text(value='', placeholder='Filter name', description='', layout=widgets.Layout(width=button_width))
apply_preset_filter_button = widgets.Button(description="Load Preset")
apply_filter_button = widgets.Button(description="Apply Filter", layout=widgets.Layout(min_width='150px'))
confirmation_filter = widgets.Output()

# Define widget creation function
def create_widget_group_filter(include_headers=False):
    headers = widgets.HBox([widgets.Label('Variable', layout=widgets.Layout(width=dropdown_width)),
                            widgets.Label('Operator', layout=widgets.Layout(width=text_input_width)),
                            widgets.Label('Value', layout=widgets.Layout(width='auto'))]
                          ) if include_headers else None

    dropdown1 = widgets.Dropdown(options=['Voc(V)', 'Jsc(mA/cm2)', 'FF(%)', 'PCE(%)', 'V_MPP(V)', 'J_MPP(mA/cm2)'],
                                 description='', layout=widgets.Layout(width=dropdown_width))
    dropdown2 = widgets.Dropdown(options=['>', '>=', '<', '<=', '==', '!='], description='', layout=widgets.Layout(width=dropdown_width))
    text_input = widgets.Text(value='', placeholder='Write a value', description='', layout=widgets.Layout(width=text_input_width))
    row = widgets.HBox([dropdown1, dropdown2, text_input])
    return widgets.VBox([headers, row]) if include_headers else row

# Initialize widget groups
widget_groups_filter = [create_widget_group_filter(include_headers=False)]
groups_container_filter = widgets.VBox(widget_groups_filter)

# Define callback functions
def add_widget_group_filter(b):
    widget_groups_filter.append(create_widget_group_filter())
    groups_container_filter.children = widget_groups_filter

def remove_widget_group_filter(b):
    if len(widget_groups_filter) > 1:
        widget_groups_filter.pop()
        groups_container_filter.children = widget_groups_filter

def save_data_filter(b):
    data = []
    for group in widget_groups_filter:
        group_data = (group.children[0].value, group.children[1].value, group.children[2].value)
        data.append(group_data)
    print("Saved data:", data, save_name_filter.value)

def apply_preset_filter(b):
    selected_preset = preset_dropdown_filter.label
    widget_groups_filter.clear()  # Clear existing widget groups before repopulating
    if selected_preset in filter_presets:  # Check if a valid preset is selected
        for variable, operator, value in filter_presets[selected_preset]:
            group = create_widget_group_filter()
            group.children[0].value = variable
            group.children[1].value = operator
            group.children[2].value = value
            widget_groups_filter.append(group)
    else:  # If "Select a preset" or an invalid option is selected
        widget_groups_filter.append(create_widget_group_filter(include_headers=False))
    groups_container_filter.children = widget_groups_filter  # Update the display

# Initialize confirmation output
with confirmation_filter:
    print(f"{warning_sign} No filter {warning_sign}")

def apply_filtering(b):
    filter_values = []
    main_output_content.clear_output()
    for group in widget_groups_filter:
        # Extract the values from each dropdown and text input in the group
        variable = group.children[0].value
        operator = group.children[1].value
        value = group.children[2].value
        filter_values.append((variable, operator, value))
    confirmation_filter.clear_output()
    with confirmation_filter:
        print("Filter Applied")
    # Apply the filter
    global filter_vals
    with main_output_content:
        data['filtered'], data['junk'], filter_vals = data_filter_setup(data['jvc'], filter_values)

# Connect callbacks to buttons
add_button_filter.on_click(add_widget_group_filter)
remove_button_filter.on_click(remove_widget_group_filter)
#save_button_filter.on_click(save_data_filter)
apply_preset_filter_button.on_click(apply_preset_filter)
apply_filter_button.on_click(apply_filtering)

# Initialize with preset
apply_preset_filter(None)

# Display UI
filters_markdown = """
# 2 Select filters
Using the dropdowns below, select filters for the data you want to keep, not remove.
"""

# Organize UI components
confirm_group_filter = widgets.HBox([apply_filter_button, confirmation_filter])
#saving_group_filter = widgets.HBox([save_button_filter, save_name_filter])
controls_filter = widgets.VBox([add_button_filter, remove_button_filter, preset_dropdown_filter, apply_preset_filter_button, confirm_group_filter])#saving_group_filter,])
#display(widgets.HBox([controls_filter, groups_container_filter, main_output_content]))

# Create a Grid layout instead  
grid_layout = widgets.GridspecLayout(1, 3)  
grid_layout[0, 0] = controls_filter  
grid_layout[0, 1] = groups_container_filter  
grid_layout[0, 2] = main_output_content  

# Display components separately  
display(Markdown(filters_markdown))  
display(grid_layout)


# 2 Select filters
Using the dropdowns below, select filters for the data you want to keep, not remove.


GridspecLayout(children=(VBox(children=(Button(description='Add Filter', style=ButtonStyle()), Button(descript…

In [5]:
%matplotlib ipympl
#%load_ext autoreload
%autoreload 2
#!pip install ipympl
from main import plotting_string_action, save_full_data_frame
import matplotlib.pyplot as plt
import shutil
import tempfile
import os
import base64
import io
import zipfile
from IPython.display import FileLink, display, HTML, Markdown


# Placeholder for presets data structure
plot_presets = {
    "Default": [("Boxplot", "PCE", "by Variable"), ("Boxplot", "Voc", "by Variable"), ("Boxplot", "Jsc", "by Variable"),
                ("Boxplot", "FF", "by Variable"), ("JV Curve", "Best device only", "")],
    "Preset 2": [("Boxplot", "Voc", "by Cell"), ("Histogram", "Voc", ""), ("JV Curve", "Best device only", "")]
}

plot_markdown = f"""
# 3 Select plots
Using the dropdowns below, select the plots you want to create.
"""

plotted_content = widgets.Output()
download_content = widgets.Output()

# Function to update additional options based on plot type selection
def update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown):
    plot_type = plot_type_dropdown.value
    if plot_type == 'Boxplot':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction']
    elif plot_type == 'Boxplot (omitted)':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['by Batch', 'by Variable', 'by Sample', 'by Cell', 'by Scan Direction']
    elif plot_type == 'Histogram':
        option1_dropdown.options = ['Voc', 'Jsc', 'FF', 'PCE', 'R_ser', 'R_shu', 'V_mpp', 'J_mpp', 'P_mpp']
        option2_dropdown.options = ['']  # Histogram does not use the second option
    elif plot_type == 'JV Curve':
        option1_dropdown.options = ['All cells', 'Only working cells', 'Only not working cells', 'Best device only', 'Separated by cell', 'Separated by substrate']
        option2_dropdown.options = ['']  # JV Curve does not use the second option
    else:
        option1_dropdown.options = []
        option2_dropdown.options = []

# Function to create a new plot type group
def create_plot_type_group():
    plot_type_dropdown = widgets.Dropdown(options=['Boxplot', 'Boxplot (omitted)', 'Histogram', 'JV Curve'], description='Plot Type:', layout=widgets.Layout(width='150px'))
    option1_dropdown = widgets.Dropdown(description='Option 1:', layout=widgets.Layout(width='150px'))
    option2_dropdown = widgets.Dropdown(description='Option 2:', layout=widgets.Layout(width='150px'))

    # Initial update for default selection
    update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown)

    plot_type_dropdown.observe(lambda change: update_additional_options(plot_type_dropdown, option1_dropdown, option2_dropdown), names='value')

    return widgets.HBox([plot_type_dropdown, option1_dropdown, option2_dropdown])

plot_type_groups = [create_plot_type_group()]
groups_container = widgets.VBox(plot_type_groups)

def add_plot_type_group(b):
    new_group = create_plot_type_group()
    plot_type_groups.append(new_group)
    groups_container.children = tuple(plot_type_groups)

def remove_plot_type_group(b):
    if len(plot_type_groups) > 1:
        plot_type_groups.pop()
        groups_container.children = tuple(plot_type_groups)

def load_preset_plot(b):
    selected_preset = preset_dropdown_plot.value
    plot_type_groups.clear()
    if selected_preset in plot_presets:
        for plot_type, option1, option2 in plot_presets[selected_preset]:
            new_group = create_plot_type_group()
            new_group.children[0].value = plot_type  # Set plot type
            new_group.children[1].value = option1    # Set option 1
            new_group.children[2].value = option2    # Set option 2
            plot_type_groups.append(new_group)
    else:  # Fallback if an invalid preset is selected
        plot_type_groups.append(create_plot_type_group())
    groups_container.children = tuple(plot_type_groups)

def plot_selected_options(b):
    extracted_contents = []
    for group in plot_type_groups:
        # Accessing the dropdown widgets directly by their index in the children tuple
        plot_type_widget, option1_widget, option2_widget = group.children[0], group.children[1], group.children[2]
        plot_type = plot_type_widget.value
        option1 = option1_widget.value
        option2 = option2_widget.value
        extracted_contents.append((plot_type, option1, option2))
    call_plotting_action(extracted_contents)
    return extracted_contents

global_plot_data = {'figs': [], 'names': [], 'workbook': None}
def center_plot(fig):
    """Center the plot on the data"""
    for ax in fig.axes:
        # Skip if this is a colorbar or other non-data axis
        if not hasattr(ax, 'lines') and not hasattr(ax, 'collections'):
            continue

        # Get all x and y data from the plot
        x_data = []
        y_data = []

        # Extract data from lines
        if hasattr(ax, 'lines'):
            for line in ax.lines:
                x_data.extend(line.get_xdata())
                y_data.extend(line.get_ydata())

        # Extract data from scatter plots, bar charts, etc.
        if hasattr(ax, 'collections'):
            for collection in ax.collections:
                if hasattr(collection, 'get_offsets'):
                    offsets = collection.get_offsets()
                    if len(offsets) > 0:
                        x_data.extend(offsets[:, 0])
                        y_data.extend(offsets[:, 1])

        # Extract data from bar plots
        if hasattr(ax, 'patches'):
            for patch in ax.patches:
                bbox = patch.get_bbox()
                x_data.extend([bbox.x0, bbox.x1])
                y_data.extend([bbox.y0, bbox.y1])

        # If we have data, center the plot
        if x_data and y_data:
            x_min, x_max = min(x_data), max(x_data)
            y_min, y_max = min(y_data), max(y_data)

            # Add a small margin (10%)
            x_margin = (x_max - x_min) * 0.1
            y_margin = (y_max - y_min) * 0.1

            # Set the limits
            ax.set_xlim(x_min - x_margin, x_max + x_margin)
            ax.set_ylim(y_min - y_margin, y_max + y_margin)

    # Redraw the figure
    fig.canvas.draw_idle()

# Store original home functions to avoid multiple overrides
original_home_functions = {}

def override_home_button(fig):
    """Override the home button for a specific figure"""
    if not hasattr(fig, 'canvas') or not hasattr(fig.canvas, 'toolbar'):
        return

    toolbar = fig.canvas.toolbar
    if not hasattr(toolbar, 'home'):
        return

    # Store the figure id to use as a key
    fig_id = id(fig)

    # Only override if we haven't already
    if fig_id not in original_home_functions:
        # Store the original home function
        original_home_functions[fig_id] = toolbar.home

        # Create a new home function that centers the plot
        def new_home(self, *args, **kwargs):
            center_plot(fig)

        # Replace the home function
        toolbar.home = new_home.__get__(toolbar, type(toolbar))

def call_plotting_action(plot_list):
    global global_plot_data

    plotted_content.clear_output()
    voila = True
    wb = save_full_data_frame('', data['jvc'])

    data_plot = data['filtered'], data["jvc"], data["curves"]
    try:
        p_rel
    except NameError:
        p_rel = os.getcwd()
    supp_plot = data['junk'], filter_vals, is_conditions, p_rel, data["curves"]["sample"].unique().tolist()

    figs, names, workbook = plotting_string_action(plot_list, wb, data_plot, supp_plot, voila)

    # Store the outputs in the global variable
    global_plot_data['figs'] = figs
    global_plot_data['names'] = names
    global_plot_data['workbook'] = workbook

    with plotted_content:
        for fig in figs:
            # Override the home button functionality before showing
            override_home_button(fig)
            fig.show()

        # Add a center all button
        center_all_button = widgets.Button(
            description='Center All Plots',
            tooltip='Click to center all plots on their data'
        )

        def on_center_all_clicked(b):
            for fig in global_plot_data['figs']:
                center_plot(fig)

        center_all_button.on_click(on_center_all_clicked)

        display(Markdown(save_markdown))
        display(widgets.HBox([save_plots_button, save_data_button, save_all_button, center_all_button]))
        display(download_content)


# Saving information
save_markdown = f"""
# 4 Save plots and data
"""

save_plots_button = widgets.Button(
    description='Save All Plots',
    tooltip='Click to save all plots',
)

save_data_button = widgets.Button(
    description='Save Data',
    tooltip='Click to save the collected raw data',
)

download_output = widgets.Output()

save_all_button = widgets.Button(
    description='Save Data & Plots',
    tooltip='Click to save the collected raw data and all plots',
)

def save_figures_to_files(figures, names, base_filepath):
    os.makedirs(base_filepath, exist_ok=True)
    for fig, name in zip(figures, names):
        filename = os.path.join(base_filepath, f"{name}.png")
        fig.savefig(filename)


def save_workbook_to_file(workbook, name, base_filepath):
    os.makedirs(base_filepath, exist_ok=True)
    filename = os.path.join(base_filepath, f"{name}")
    workbook.save(filename)

# New JavaScript-based download functions
def on_save_plots_button_clicked(b):
    download_content.clear_output()

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for fig, name in zip(global_plot_data['figs'], global_plot_data['names']):
            img_buffer = io.BytesIO()
            fig.savefig(img_buffer, format='png')
            img_buffer.seek(0)
            zip_file.writestr(f"{name}.png", img_buffer.getvalue())

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'plots.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_content:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all plots</button>"))

def on_save_data_button_clicked(b):
    download_content.clear_output()

    # Save Excel file to memory
    excel_buffer = io.BytesIO()
    global_plot_data['workbook'].save(excel_buffer)
    excel_buffer.seek(0)

    # Encode to base64
    b64 = base64.b64encode(excel_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}';
    link.download = 'collected_data.xlsx';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_content:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download data</button>"))

def on_save_all_button_clicked(b):
    download_content.clear_output()

    # Create a zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
        # Add figures to zip
        for fig, name in zip(global_plot_data['figs'], global_plot_data['names']):
            img_buffer = io.BytesIO()
            fig.savefig(img_buffer, format='png')
            img_buffer.seek(0)
            zip_file.writestr(f"{name}.png", img_buffer.getvalue())

        # Add Excel file to zip
        excel_buffer = io.BytesIO()
        global_plot_data['workbook'].save(excel_buffer)
        excel_buffer.seek(0)
        zip_file.writestr("collected_data.xlsx", excel_buffer.getvalue())

    # Create download link
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()

    # JavaScript to trigger download
    js_code = f"""
    var link = document.createElement('a');
    link.href = 'data:application/zip;base64,{b64}';
    link.download = 'results.zip';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """

    with download_content:
        display(HTML(f"<button onclick=\"{js_code}\">Click to download all files</button>"))

# Attach the event handler to the button
save_plots_button.on_click(on_save_plots_button_clicked)
save_data_button.on_click(on_save_data_button_clicked)
save_all_button.on_click(on_save_all_button_clicked)

add_button_plot = widgets.Button(description="Add Plot Type")
remove_button_plot = widgets.Button(description="Remove Plot Type")
load_preset_plot_button = widgets.Button(description="Load Preset")
preset_dropdown_plot = widgets.Dropdown(options=list(plot_presets.keys()), description='Presets')
plot_selection_button = widgets.Button(description="Plot selection",layout=widgets.Layout(min_width='150px'))

add_button_plot.on_click(add_plot_type_group)
remove_button_plot.on_click(remove_plot_type_group)
load_preset_plot_button.on_click(load_preset_plot)
plot_selection_button.on_click(plot_selected_options)

load_preset_plot(None)

display(Markdown(plot_markdown))
controls_plot = widgets.VBox([add_button_plot, remove_button_plot, preset_dropdown_plot, load_preset_plot_button, plot_selection_button])
display(widgets.HBox([controls_plot, groups_container]))
display(plotted_content)


# 3 Select plots
Using the dropdowns below, select the plots you want to create.


Output()